Este notebook demonstra:

1. Geração de dados fictícios
2. Pré-processamento e treinamento do modelo KNN
3. Avaliação de precisão@K e recall@K


## 1. Configurações e importações

In [1]:
import random
import pandas as pd
from scipy.sparse import csr_matrix
import logging
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import normalize

# === Configurações do modelo ===
K_VIZINHOS = 20  # Quantidade de vizinhos similares a considerar
K_RECS = 10  # Número de itens a recomendar
ALPHA = 0.01  # Taxa de decaimento para ponderação de recência
MIN_PRODUCT_SUPPORT = 5  # Número mínimo de clientes que compraram um produto
MIN_CLIENT_TRANSACTIONS = 5  # Número mínimo de transações por cliente
MIN_QUANTITY = 5  # Quantidade mínima para considerar interação válida
# === Configurações de geração ===
NUM_ORDERS_PER_CLIENT = 100  # Quantidade de pedidos por cliente
MAX_DATE_OFFSET_DAYS = 365  # Intervalo máximo para datas futuras
OUTPUT_CSV_PATH = "data/sells_data.csv"  # Caminho de saída

# === Definição de entidades ===
# (Listas completas omitidas para brevidade)
CLIENTS = [  # lista de nomes de clientes
    "Ana",
    "Beatriz",
    "Camila",
    "Daniela",
    "Elisa",
    "Fernanda",
    "Gabriela",
    "Heloísa",
    "Isabela",
    "Júlia",
    "Juliana",
    "Larissa",
    "Mariana",
    "Natália",
    "Olívia",
    "Patrícia",
    "Rafaela",
    "Sabrina",
    "Tatiane",
    "Vanessa",
    "Yasmin",
    "Mônica",
    "Débora",
    "Cíntia",
    "Érica",
    "André",
    "Bruno",
    "Caio",
    "Diego",
    "Eduardo",
    "Felipe",
    "Gabriel",
    "Henrique",
    "Igor",
    "João",
    "Lucas",
    "Leandro",
    "Marcelo",
    "Nicolas",
    "Otávio",
    "Paulo",
    "Rafael",
    "Sérgio",
    "Tiago",
    "Vinícius",
    "Alexandre",
    "Gustavo",
    "Matheus",
    "Pedro",
    "Rodrigo",
]

PRODUCTS = [  # lista de produtos disponíveis
    "Abacate",
    "Abacaxi",
    "Abóbora",
    "Abóbora italiana",
    "Abóbora japonesa - Tetsukabut",
    "Abóbora menina",
    "Acerola",
    "Agrião",
    "Alface",
    "Alho",
    "Atemóia",
    "Banana",
    "Batata",
    "Batata-doce",
    "Berinjela",
    "Beterraba",
    "Brazlândia",
    "Brócolis - Cabeça Única",
    "Brócolis - Ramoso",
    "Cajamanga",
    "Caqui",
    "Cebola",
    "Cebolinha",
    "Ceilândia",
    "Cenoura",
    "Chuchu",
    "Coco",
    "Coentro",
    "Couve",
    "Couve-flor",
    "Gama",
    "Gengibre",
    "Goiaba",
    "Graviola",
    "Jabuticaba",
    "Jardim",
    "Jiló",
    "Laranja",
    "Lichia",
    "Limão",
    "Mamão",
    "Mandioca",
    "Manga",
    "Maracujá",
    "Maracujá Pérola",
    "Milho doce",
    "Milho-verde",
    "Mirtilo",
    "Morango",
    "PAD-DF",
    "Paranoá",
    "Pepino",
    "Pimentão",
    "Pipiripau",
    "Pitaia",
    "Planaltina",
    "Quiabo",
    "Repolho",
    "Rio Preto",
    "Sobradinho",
    "São Sebastião",
    "Tabatinga",
    "Tangerina",
    "Taquara",
    "Tomate",
    "Uva",
    "Uva Vinífera",
    "Vargem Bonita",
]

ORIGINAL_LOCATIONS = [  # lista de localidades
    "Alexandre Gusmão",
    "Brazlândia",
    "Ceilândia",
    "Gama",
    "Jardim",
    "PAD-DF",
    "Paranoá",
    "Pipiripau",
    "Planaltina",
    "Rio Preto",
    "Sobradinho",
    "São Sebastião",
    "Tabatinga",
    "Taquara",
    "Vargem Bonita",
]

LOCATIONS_PRODUCTS = {  # mapeia cada localidade a seus produtos típicos
    "Alexandre Gusmão": [
        "Goiaba",
        "Abacate",
        "Tangerina",
        "Limão",
        "Banana",
        "Graviola",
        "Maracujá",
        "Manga",
        "Atemóia",
        "Uva",
        "Brazlândia",
        "Alface",
        "Mandioca",
        "Brócolis - Cabeça Única",
        "Repolho",
        "Couve",
        "Cebolinha",
        "Tomate",
        "Chuchu",
        "Brócolis - Ramoso",
        "Pimentão",
        "Brazlândia",
    ],
    "Brazlândia": [
        "Goiaba",
        "Abacate",
        "Limão",
        "Tangerina",
        "Banana",
        "Atemóia",
        "Maracujá",
        "Manga",
        "Uva",
        "Cajamanga",
        "Ceilândia",
        "Morango",
        "Alface",
        "Repolho",
        "Abóbora italiana",
        "Mandioca",
        "Brócolis - Cabeça Única",
        "Chuchu",
        "Tomate",
        "Pimentão",
        "Couve",
        "Ceilândia",
    ],
    "Ceilândia": [
        "Banana",
        "Limão",
        "Abacate",
        "Tangerina",
        "Maracujá",
        "Manga",
        "Laranja",
        "Goiaba",
        "Coco",
        "Acerola",
        "Gama",
        "Alface",
        "Mandioca",
        "Milho-verde",
        "Brócolis - Cabeça Única",
        "Repolho",
        "Brócolis - Ramoso",
        "Batata-doce",
        "Chuchu",
        "Couve",
        "Tomate",
        "Gama",
    ],
    "Gama": [
        "Limão",
        "Abacate",
        "Banana",
        "Tangerina",
        "Manga",
        "Acerola",
        "Pitaia",
        "Maracujá",
        "Laranja",
        "Lichia",
        "Jardim",
        "Alface",
        "Mandioca",
        "Milho-verde",
        "Coentro",
        "Brócolis - Cabeça Única",
        "Couve",
        "Cebolinha",
        "Brócolis - Ramoso",
        "Repolho",
        "Pimentão",
        "Jardim",
    ],
    "Jardim": [
        "Maracujá",
        "Uva Vinífera",
        "Limão",
        "Tangerina",
        "Graviola",
        "Banana",
        "Abacate",
        "Goiaba",
        "Mamão",
        "PAD-DF",
        "Cenoura",
        "Cebola",
        "Abóbora japonesa - Tetsukabut",
        "Tomate",
        "Beterraba",
        "Batata-doce",
        "Mandioca",
        "Repolho",
        "Pimentão",
        "Berinjela",
        "PAD-DF",
    ],
    "PAD-DF": [
        "Uva Vinífera",
        "Uva",
        "Abacate",
        "Limão",
        "Maracujá",
        "Banana",
        "Manga",
        "Tangerina",
        "Mamão",
        "Laranja",
        "Paranoá",
        "Tomate",
        "Milho doce",
        "Cenoura",
        "Cebola",
        "Pepino",
        "Mandioca",
        "Alface",
        "Pimentão",
        "Repolho",
        "Berinjela",
        "Paranoá",
    ],
    "Paranoá": [
        "Banana",
        "Limão",
        "Manga",
        "Uva",
        "Abacate",
        "Tangerina",
        "Goiaba",
        "Maracujá",
        "Laranja",
        "Maracujá Pérola",
        "Pipiripau",
        "Alface",
        "Milho-verde",
        "Mandioca",
        "Beterraba",
        "Tomate",
        "Quiabo",
        "Pimentão",
        "Repolho",
        "Berinjela",
        "Chuchu",
        "Pipiripau",
    ],
    "Pipiripau": [
        "Laranja",
        "Banana",
        "Limão",
        "Lichia",
        "Goiaba",
        "Abacate",
        "Pitaia",
        "Maracujá",
        "Tangerina",
        "Uva",
        "Planaltina",
        "Batata",
        "Mandioca",
        "Tomate",
        "Pimentão",
        "Abóbora",
        "Cenoura",
        "Pepino",
        "Abóbora italiana",
        "Berinjela",
        "Chuchu",
        "Planaltina",
    ],
    "Planaltina": [
        "Limão",
        "Banana",
        "Lichia",
        "Abacate",
        "Tangerina",
        "Manga",
        "Uva",
        "Maracujá",
        "Laranja",
        "Goiaba",
        "Rio Preto",
        "Milho-verde",
        "Mandioca",
        "Batata-doce",
        "Tomate",
        "Alface",
        "Cebolinha",
        "Repolho",
        "Pimentão",
        "Chuchu",
        "Couve",
        "Rio Preto",
    ],
    "Rio Preto": [
        "Abacate",
        "Tangerina",
        "Banana",
        "Limão",
        "Maracujá",
        "Jabuticaba",
        "Manga",
        "Pitaia",
        "Graviola",
        "Laranja",
        "São Sebastião",
        "Alho",
        "Mandioca",
        "Abóbora japonesa - Tetsukabut",
        "Batata-doce",
        "Beterraba",
        "Abóbora",
        "Tomate",
        "Pimentão",
        "Cenoura",
        "Repolho",
        "São Sebastião",
    ],
    "São Sebastião": [
        "Maracujá",
        "Manga",
        "Banana",
        "Limão",
        "Maracujá Pérola",
        "Tangerina",
        "Abacate",
        "Laranja",
        "Abacaxi",
        "Pitaia",
        "Sobradinho",
        "Cebola",
        "Mandioca",
        "Alface",
        "Abóbora",
        "Morango",
        "Quiabo",
        "Couve",
        "Tomate",
        "Agrião",
        "Repolho",
        "Sobradinho",
    ],
    "Sobradinho": [
        "Banana",
        "Abacate",
        "Manga",
        "Limão",
        "Uva",
        "Tangerina",
        "Maracujá",
        "Mirtilo",
        "Laranja",
        "Maracujá Pérola",
        "Tabatinga",
        "Mandioca",
        "Alface",
        "Repolho",
        "Brócolis - Ramoso",
        "Tomate",
        "Couve",
        "Couve-flor",
        "Berinjela",
        "Pimentão",
        "Chuchu",
        "Tabatinga",
    ],
    "Tabatinga": [
        "Abacate",
        "Limão",
        "Maracujá",
        "Tangerina",
        "Banana",
        "Goiaba",
        "Maracujá Pérola",
        "Pitaia",
        "Manga",
        "Uva",
        "Taquara",
        "Batata-doce",
        "Mandioca",
        "Cenoura",
        "Cebola",
        "Abóbora japonesa - Tetsukabut",
        "Beterraba",
        "Jiló",
        "Repolho",
        "Chuchu",
        "Tomate",
        "Taquara",
    ],
    "Taquara": [
        "Banana",
        "Tangerina",
        "Abacate",
        "Maracujá",
        "Limão",
        "Laranja",
        "Manga",
        "Pitaia",
        "Caqui",
        "Acerola",
        "Vargem Bonita",
        "Mandioca",
        "Abóbora menina",
        "Pimentão",
        "Tomate",
        "Abóbora italiana",
        "Berinjela",
        "Pepino",
        "Couve-flor",
        "Repolho",
        "Chuchu",
        "Vargem Bonita",
    ],
    "Vargem Bonita": [
        "Banana",
        "Abacate",
        "Limão",
        "Manga",
        "Uva",
        "Pitaia",
        "Tangerina",
        "Maracujá",
        "Maracujá Pérola",
        "Laranja",
        "Alface",
        "Mandioca",
        "Milho-verde",
        "Gengibre",
        "Couve",
        "Brócolis - Cabeça Única",
        "Repolho",
        "Cebolinha",
        "Brócolis - Ramoso",
        "Pimentão",
    ],
}

# Opções de feedback possíveis
FEEDBACK_OPTIONS = ["Excelente", "Bom", "Regular", "Ruim", "Péssimo"]

## 2. Geração de dados fictícios

Função `generate_fake_sales`

In [2]:
def generate_fake_sales() -> list[dict]:
    """
    Gera registros de vendas fictícias:
    - Cada cliente recebe NUM_ORDERS_PER_CLIENT entradas.
    - Produtos com histórico ou típicos da localidade têm maior probabilidade.
    - Quantidades são maiores para produtos regionais.
    - Datas são offset aleatório até MAX_DATE_OFFSET_DAYS dias.

    Retorna:
        Listagem de dicionários prontos para DataFrame.
    """
    sells = []
    base_date = pd.Timestamp.now().normalize()

    for idx, client in enumerate(CLIENTS):
        # Atribui localidade de forma cíclica
        location = ORIGINAL_LOCATIONS[idx % len(ORIGINAL_LOCATIONS)]

        # Histórico de produtos comprados para o cliente
        client_history = []

        for _ in range(NUM_ORDERS_PER_CLIENT):
            # Calcula pesos para escolha de produto
            weights = [
                100
                if p in client_history
                else 25
                if p in LOCATIONS_PRODUCTS.get(location, [])
                else 1
                for p in PRODUCTS
            ]
            product = random.choices(PRODUCTS, weights=weights, k=1)[0]

            # Define quantidade com base em regionalidade
            if product in LOCATIONS_PRODUCTS.get(location, []):
                quantity = random.randint(50, 100)
            else:
                quantity = random.randint(1, 10)

            # Simula data com deslocamento aleatório
            date = (
                base_date
                + pd.Timedelta(days=random.randint(0, MAX_DATE_OFFSET_DAYS))
                + pd.Timedelta(hours=random.randint(0, 23))
                + pd.Timedelta(minutes=random.randint(0, 59))
            )

            # Cria registro e atualiza histórico
            record = {
                "client": client,
                "location": location,
                "product": product,
                "quantity": quantity,
                "customerFeedback": random.choice(FEEDBACK_OPTIONS),
                "date": date,
            }
            sells.append(record)
            client_history.append(product)

    return sells


if __name__ == "__main__":
    # Gera dados e salva em CSV
    df_sales = pd.DataFrame(generate_fake_sales())
    df_sales.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")
    print(f"Geração de dados concluída. Arquivo salvo em '{OUTPUT_CSV_PATH}'.")


Geração de dados concluída. Arquivo salvo em 'data/sells_data.csv'.


## 3. Pré-processamento e treinamento do modelo

In [3]:
# === Configurações do modelo ===
K_VIZINHOS = 20  # Quantidade de vizinhos similares a considerar
K_RECS = 10  # Número de itens a recomendar
ALPHA = 0.01  # Taxa de decaimento para ponderação de recência
MIN_PRODUCT_SUPPORT = 5  # Número mínimo de clientes que compraram um produto
MIN_CLIENT_TRANSACTIONS = 5  # Número mínimo de transações por cliente
MIN_QUANTITY = 5  # Quantidade mínima para considerar interação válida

# === Logging ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# === Carregamento e preparação dos dados ===
logger.info("Carregando dados de vendas")
df_comp = pd.read_csv("data/sells_data.csv")

# Filtra interações de quantidade muito baixa
logger.info(f"Removendo interações com quantity < {MIN_QUANTITY}")
df_comp = df_comp[df_comp["quantity"] >= MIN_QUANTITY]

# Converte datas e calcula peso de recência
logger.info("Convertendo datas e calculando recência")
df_comp["date"] = pd.to_datetime(df_comp["date"])
max_date = df_comp["date"].max()
df_comp["days_since"] = (max_date - df_comp["date"]).dt.days
# Peso de recência decai com o tempo
df_comp["recency_weight"] = 1 / (1 + ALPHA * df_comp["days_since"])
# Aplica o peso ao volume comprado
df_comp["weighted_quantity"] = df_comp["quantity"] * df_comp["recency_weight"]

# === Filtragem de ruído geral ===
logger.info("Filtrando produtos esparsos e clientes com poucas transações")
support = df_comp.groupby("product")["client"].nunique()
popular_products = support[support >= MIN_PRODUCT_SUPPORT].index
df_comp = df_comp[df_comp["product"].isin(popular_products)]
txn_counts = df_comp["client"].value_counts()
active_clients = txn_counts[txn_counts >= MIN_CLIENT_TRANSACTIONS].index
df_comp = df_comp[df_comp["client"].isin(active_clients)]

# === Construção da matriz de características ===
logger.info("Gerando pivot table de cliente x produto (weighted_quantity)")
pivot = df_comp.pivot_table(
    index="client", columns="product", values="weighted_quantity", aggfunc="sum", fill_value=0
)

# Mapeia feedback qualitativo para escore numérico e adiciona média por cliente
logger.info("Mapeando feedback e adicionando avg_feedback")
feedback_map = {"Excelente": 5, "Bom": 4, "Regular": 3, "Ruim": 2, "Péssimo": 1}
df_comp["feedback_score"] = df_comp["customerFeedback"].map(feedback_map)
feedback_avg = df_comp.groupby("client")["feedback_score"].mean().rename("avg_feedback")
pivot = pivot.merge(feedback_avg, left_index=True, right_index=True)

# Converte pivot para matriz densa
dense_matrix = pivot.values

# Aplica TF–IDF para ajustar importância de produtos
logger.info("Aplicando TF-IDF nas features")
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(dense_matrix)

# Normaliza vetores cliente para norma L2
logger.info("Normalizando vetores de cliente")
norm_matrix = normalize(tfidf_matrix, norm="l2", axis=1)

# Converte para matriz esparsa para o KNN
dense_norm = norm_matrix.toarray()  # OK para tamanhos moderados
df_sparse = csr_matrix(dense_norm)

# Atualiza variáveis globais
tt_clientes = pivot.index.tolist()
bursos = pivot.columns.tolist()

# === Treinamento do modelo KNN ===
logger.info("Treinando modelo NearestNeighbors")
knn = NearestNeighbors(n_neighbors=K_VIZINHOS, metric="cosine", algorithm="brute")
knn.fit(df_sparse)
logger.info("Modelo treinado com sucesso")

# === Funções de recomendação e histórico ===


def recomendar_por_cliente(client: str, k_vizinhos: int = K_VIZINHOS, k_recs: int = K_RECS) -> list:
    """
    Recomenda produtos para um cliente com base em vizinhos mais similares.

    Args:
        client: identificador do cliente.
        k_vizinhos: número de clientes vizinhos a considerar.
        k_recs: número de itens recomendados.

    Returns:
        Lista de produtos recomendados.
    """
    if client not in tt_clientes:
        logger.error(f"Cliente não encontrado: {client}")
        raise ValueError(f"Cliente '{client}' não encontrado.")

    idx = tt_clientes.index(client)
    dist, viz_idx = knn.kneighbors(df_sparse[idx], n_neighbors=k_vizinhos + 1)
    vizinhos_list = viz_idx[0].tolist()
    if idx in vizinhos_list:
        vizinhos_list.remove(idx)
    vizinhos_list = vizinhos_list[:k_vizinhos]

    scores = np.array(df_sparse[vizinhos_list].sum(axis=0)).ravel()
    top_idx = np.argsort(scores)[::-1][:k_recs]
    return [bursos[i] for i in top_idx]


def get_client_purchases(client: str) -> list[dict]:
    """
    Retorna o histórico de compras de um cliente.
    """
    if client not in tt_clientes:
        raise ValueError(f"Cliente '{client}' não encontrado.")
    df_client = df_comp[df_comp["client"] == client]
    df_sorted = df_client.sort_values("quantity", ascending=False)
    return df_sorted[["product", "quantity"]].to_dict("records")


# Exemplo de execução
if __name__ == "__main__":
    sample = tt_clientes[0] if tt_clientes else None
    if sample:
        print(recomendar_por_cliente(sample))

2025-05-28 02:42:11 - INFO - Carregando dados de vendas
2025-05-28 02:42:11 - INFO - Removendo interações com quantity < 5
2025-05-28 02:42:11 - INFO - Convertendo datas e calculando recência
2025-05-28 02:42:11 - INFO - Filtrando produtos esparsos e clientes com poucas transações
2025-05-28 02:42:11 - INFO - Gerando pivot table de cliente x produto (weighted_quantity)
2025-05-28 02:42:11 - INFO - Mapeando feedback e adicionando avg_feedback
2025-05-28 02:42:11 - INFO - Aplicando TF-IDF nas features
2025-05-28 02:42:11 - INFO - Normalizando vetores de cliente
2025-05-28 02:42:11 - INFO - Treinando modelo NearestNeighbors
2025-05-28 02:42:11 - INFO - Modelo treinado com sucesso


['Couve', 'Manga', 'Brócolis - Ramoso', 'Abacate', 'Alface', 'Banana', 'Milho-verde', 'Limão', 'Repolho', 'Tomate']


## 4. Avaliação do modelo

Implementação de `avaliar_knn_v2`

In [4]:
def avaliar_knn_v2(
    df_comp: pd.DataFrame,
    recomendar_fn,
    k_vizinhos: int = K_VIZINHOS,
    k_recs: int = K_RECS,
    test_frac: float = 0.1,
) -> dict:
    """
    Avalia o desempenho do modelo KNN considerando recência e feedback:
      - Separa uma fração de interações de cada cliente para teste.
      - Reajusta o KNN nos dados de treino (ponderados por recência e com feedback médio).
      - Gera recomendações e calcula precision@K e recall@K.
    """
    # Cópia dos dados e marcação de amostras de teste
    df = df_comp.copy()
    df["is_test"] = False
    for cliente, grp in df.groupby("client"):
        n_test = max(1, int(len(grp) * test_frac))
        idxs = grp.sample(n=n_test, random_state=42).index
        df.loc[idxs, "is_test"] = True

    # Separação em treino e teste
    df_train = df[~df["is_test"]]
    df_test = df[df["is_test"]]

    # Pivot dos dados de treino: utiliza weighted_quantity (recência) e adiciona avg_feedback
    pivot_train = df_train.pivot_table(
        index="client", columns="product", values="weighted_quantity", aggfunc="sum", fill_value=0
    )
    feedback_avg = df_train.groupby("client")["feedback_score"].mean().rename("avg_feedback")
    pivot_train = pivot_train.merge(feedback_avg, left_index=True, right_index=True)

    # Converte para matriz esparsa e refaz o KNN
    clientes = pivot_train.index.to_list()
    produtos = pivot_train.columns.to_list()
    mat_train = csr_matrix(pivot_train.values)
    knn.fit(mat_train)

    # Atualiza globais usados por recomendar_por_cliente
    global pivot, localidades, itens, mat_sparse
    pivot = pivot_train
    localidades = clientes
    itens = produtos
    mat_sparse = mat_train

    precisions, recalls = [], []
    # Avaliação de cada cliente no teste
    for cliente in df_test["client"].unique():
        itens_test = df_test.loc[df_test["client"] == cliente, "product"].unique().tolist()
        if not itens_test:
            continue
        recs = recomendar_fn(cliente, k_vizinhos=k_vizinhos, k_recs=k_recs)
        hits = set(recs) & set(itens_test)
        precisions.append(len(hits) / k_recs)
        recalls.append(len(hits) / len(itens_test))

    return {
        "precision@K": float(np.mean(precisions)) if precisions else 0.0,
        "recall@K": float(np.mean(recalls)) if recalls else 0.0,
    }

In [5]:
if __name__ == "__main__":
    metrics = avaliar_knn_v2(df_comp, recomendar_por_cliente)
    print(metrics)

{'precision@K': 0.36199999999999993, 'recall@K': 0.5422301587301587}


## Conclusão

Este notebook executou uma única iteração do testbench: geração de dados, treinamento e avaliação do modelo KNN para recomendações.
Agora você pode utilizar o `grid_search.py` para encontrar os melhores parâmetros e depois de introduzi-los no `ai.py`, executar o `testbench.py` para encontrar o melhor caso-base.